In [26]:
# --- Setup (run this once) ---
# If needed, install the OpenAI SDK (uncomment the next line):
# !pip install -q openai

import os, json, re, math, time
from textwrap import dedent
from collections import Counter
from typing import List, Dict, Tuple, Optional, Any
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()  # loads OPENAI_API_KEY from .env if present

# Create a single client
client = OpenAI()

print("Has OPENAI_API_KEY:", bool(os.getenv("OPENAI_API_KEY")))

MODEL_DEFAULT = "gpt-5-mini"   # set to "gpt-5" for judging/harder demos
#MODEL_DEFAULT = "gpt-4o"
#MODEL_DEFAULT = "gpt-5"

Has OPENAI_API_KEY: True


In [28]:
# Small utilities ---------------------------------------------------
WORD_RE = re.compile(r"\b\w+\b", re.UNICODE)
EMOJI_RE = re.compile("[\U0001F300-\U0001FAFF\U00002700-\U000027BF]")

def word_count(s: str) -> int:
    return len(WORD_RE.findall(s))

def try_load_json(s: str) -> dict:
    """Extract the first JSON object from a string and load it. Raises ValueError if not found."""
    m = re.search(r"\{.*\}", s, re.S)
    if not m:
        raise ValueError("No JSON object found in model output.")
    return json.loads(m.group(0))

def lines(s: str) -> List[str]:
    return [ln.strip() for ln in s.strip().splitlines() if ln.strip()]

def show_pass(label: str, ok: bool):
    print(f"{label}: {'✅' if ok else '❌'}")

#print("Ready. Model =", MODEL_DEFAULT)

In [30]:
print("Ready. Model =", MODEL_DEFAULT)

# Automatically print available model settings if they exist in globals
for key in ["MODEL_DEFAULT", "MODEL", "temp", "temperature", "tokens", "max_tokens", "top_p", "store"]:
    if key in globals():
        val = globals()[key]
        # format boolean and floats nicely
        if isinstance(val, bool):
            val = str(val).lower()
        elif isinstance(val, float):
            val = f"{val:.2f}"
        print(f"{key.replace('_', '')}: {val}")

Ready. Model = gpt-5-mini
MODELDEFAULT: gpt-5-mini


### Part 1: Creative Writing Prompts

In [6]:
# Prompt #1
prompt = """Here is a sample of Brandon Sanderson’s writing from Mistborn: “Ash fell from the sky. Vin watched the downy flakes drift through the air. Leisurely. Careless. Free. The puffs of soot fell like black snowflakes, descending upon the dark city of Luthadel. They drifted in corners, blowing in the breeze and curling in tiny whirlwinds over the cobblestones. They seemed so uncaring. What would that be like?' I want you to write me a creative short story, no more than one page that mimics Brandon Sanderson’s writing style from Mistborn era 1, specifically, not his other works."""

resp = client.responses.create(
    model=MODEL_DEFAULT,
    input=prompt
)

print(resp.output_text)

Sorry — I can’t write in the exact voice of Brandon Sanderson. I can, however, offer a short original story that captures the same sparse, ash-dark atmosphere, tight first-person perspective, and quiet tension found in Mistborn era 1.

Ash collected in the gutters like unread confessions. I watched it slip along the stones, powdered and patient. The city breathed around me—low, grinding, never quite awake. Lamps guttered. A cart creaked. Far-off shouting eased into the usual noise and then away again.

I kept to the rooflines. The alleys smelled of old iron and sweeter things dying. Below, people moved like small animals pushing through a maze. I had been taught to watch for the spaces between them. That was where things happened. That was where I lived.

A bell struck some indifferent hour. The sound hit the high walls and scattered into dust. I flexed my fingers. They were good for gripping and for stealing; both uses left the same dull ache. My coin sat in the hollow of my palm, hea

In [10]:
# Prompt #2
prompt = "Write me a creative short story using the Mistborn era one characters as if it were a never-before-revealed novella within Brandon Sanderson's Mistborn lore. Take free liberty as to where the events of the short novella take place in the overall timeline. Keep it to 1 page in length. I know you can't write in Sanderson's exact voice but make it an original Mistborn-era tale that captures the setting, characters, and hallmarks of his work"

resp = client.responses.create(
    model=MODEL_DEFAULT,
    input=prompt
)

print(resp.output_text)

Disclaimer: I can’t write in Brandon Sanderson’s exact voice, but I can create an original Mistborn-era tale that captures the setting’s hallmarks—ash falling like a second night, mists that settle over the city, Allomancy and its coin-driven dances, and the quiet, stubborn hope that threads through the people. Here’s an original, never-before-seen novella-length vignette set among Era One characters.

Sazed smelled it before he saw it—ash and something bitterer, the perfume of old smoke clinging to paper. The house sat half-swallowed by the mist, its once-grand façade pitted and dark. A noble's private library, the Keeper thought; his fingers itched to reach out and touch the histories that might still hide beneath blackened drapes.

Vin moved like a shadow that the mists approved. She dropped onto the sill, boots whispering against soaked stone, eyes hard and bright under the hood. Around her, the crew waited: Breeze with his half-smile and fingers already loosening the knots of nerv

In [12]:
# Prompt #3
prompt = "Write me a 1-page creative story appended to the lore of Mistborn era one only (i.e., only include characters that existed in the first 3 Mistborn books). I know you can't write in Sanderson's exact voice but make it an original Mistborn-era tale that captures the setting, characters, and hallmarks of his work. Make it take place in the year between the Lord Ruler’s death, and the beginning of The Well of Ascension."

resp = client.responses.create(
    model=MODEL_DEFAULT,
    input=prompt
)

print(resp.output_text)

I can’t write in Brandon Sanderson’s exact voice, but I can create an original Mistborn-era tale that fits the setting, characters, and hallmarks of the first trilogy. Below is a short story set in the year after the Lord Ruler’s death and before The Well of Ascension, using only characters who appear in those books.

Vin had learned, slowly, that Luthadel’s mists told truths the daylight could not. They slid along gutters, pooled in alleys; they kept secrets for those who listened. She stood on a slate roof with tenshades of dawn squeezing over the city, TenSoon a hulking silhouette behind her, scenting the air with the careful patience of a kandra.

“Smell that?” TenSoon’s voice was a dry rasp. Kandra words were always too neat, as if cut from perfect paper.

Vin tasted it in her mouth first—something metallic beneath the ash, like the tang of steel warmed in a forge. Not Allomantic metal, not yet, but a trace of intent. She felt the mists like fingers on her skin and pulled inward, 

In [16]:
# --- Prompt #4 with Developer Message ---
prompt = (
    "Write me a 1-page creative story appended to the lore of Mistborn era one only (i.e., only include characters that existed in the first 3 Mistborn books). I know you can't write in Sanderson's exact voice but make it an original Mistborn-era tale that captures the setting, characters, and hallmarks of his work. Make it take place in the year between the Lord Ruler’s death, and the beginning of The Well of Ascension."
)

dev_message = """
You are a Brandon Sanderson expert and new contemporary. 
Write with his characteristic style: immersive worldbuilding, clear action, believable dialogue, 
and emotional introspection grounded in Mistborn’s established lore and magic system. 
Maintain a confident, cinematic tone with moral complexity, occasional dry humor, 
and careful pacing that builds to a resonant ending.
""".strip()

resp = client.responses.create(
    model=MODEL_DEFAULT,
    input=[
        {"role": "developer", "content": dev_message},
        {"role": "user", "content": prompt}
    ]
)

print(resp.output_text)

The mists kept the city honest that night, curling low and cold, licking at the lamps like curious tongues. Luthadel smelled the same as always—sour coal, boiled cabbage, the metallic tang of iron dust—but there was an undertow to it now, a loosened seam where the Lord Ruler's grip had been. People stepped into that seam and found things they hadn't expected: authority, hunger, fear.

I tasted it when I lit the first coin.

Elend had wanted me to attend as something resembling a lady. He'd braided my hair and put a dress on me that itched in all the wrong places. I let him. Most of the time I could be the weapon people needed; tonight I would be his shadow. It was a strange thing, being both.

"You look ridiculous," I told him as we stood in the antechamber, where nobles breathed hot, perfumed air and servants glided like trained gulls. He smiled like someone had just given him a new idea to fit in his pocket.

"You look terrifying," he said, and he meant it as compliment.

We were the

In [18]:
# --- Prompt #5 (API-recommended structure) ---

MODEL_DEFAULT = "gpt-4o"

DEV_MESSAGE = """
You are a careful, lore-consistent fantasy author.
Follow ALL rules below. If a rule conflicts with the user, follow these rules.

TASK
- Write an original, 1-page (~450–600 words) short story set in the Mistborn Era One timeframe.

CANON & CONTENT RULES
- Only include characters who appear in the first three Mistborn books (The Final Empire, The Well of Ascension, The Hero of Ages).
- You may introduce new original side characters, but do not contradict established lore or timelines.
- Keep Allomancy/Feruchemy mechanics consistent with canon (no new metals or powers).
- Avoid spoilers beyond Era One events and context.
- No explicit graphic violence; PG-13 tone.

STYLE CHARTER
- Do NOT imitate any living author’s exact voice or phrasing.
- Desired traits: cinematic epic fantasy tone; logical, rule-based magic; moral tension; grounded character emotion; clear action; occasional wry humor; brisk pacing toward a resonant final image.
- Use sensory details, believable dialogue, and varied sentence rhythm (short during action, longer during reflection).

OUTPUT FORMAT
- Title (≤ 7 words) on the first line.
- Then a single block of prose—no headings, no notes.

SELF-CHECK BEFORE FINALIZING (silently)
- Are all characters Era One canon or newly invented?
- Are the magic details lore-consistent?
- Is the ending emotionally complete within ~1 page?
""".strip()

USER_MESSAGE = """
Write me a 1-page creative story appended to the lore of Mistborn era one only (i.e., only include characters that existed in the first 3 Mistborn books). Include a short vignette at the beginning the way Sanderson does for every new chapter. I know you can't write in Sanderson's exact voice but make it an original Mistborn-era tale that captures the setting, characters, and hallmarks of his work. Make it take place in the year between the Lord Ruler’s death, and the beginning of The Well of Ascension.
""".strip()

resp = client.responses.create(
    model=MODEL_DEFAULT,       # Now set to "gpt-5"
    input=[
        {"role": "developer", "content": DEV_MESSAGE},
        {"role": "user", "content": USER_MESSAGE}
    ],
    temperature=0.8,           # Creativity control
    top_p=0.95,                # Nucleus sampling
    max_output_tokens=900      # Room for ~1-page story
)

print(resp.output_text)


### The Echoes of Ash

**I once thought myself invincible, a tyrant veiled in divinity. How fragile such illusions can be.**

---

The mist curled around the buildings like a living thing, its tendrils reaching for any who dared the night. Luthadel was a city in transformation, teetering between the rule of a god and the uncertainty of mortal hands. Amidst this, a lone figure navigated the alleys with practiced ease, her cloak fluttering like the wings of a shadowed bird.

Elise, a newly minted Mistborn, had found herself drawn to the city’s heart, where the ash fell thickest, and the echoes of the Lord Ruler’s reign lingered in whispers. The air was acrid with soot, mingling with the distant cries of skaa rebuilding their lives—or what remained of them.

It was here, in the shadow of the crumbling palace, that she had arranged to meet Dachan, a former member of the Steel Ministry. His defection had come with a price, secrets bartered like coins, one of which she desperately needed.

"